In [40]:
import scipy
from scipy import stats
from scipy import special
import numpy as np
from numpy import random
import pandas as pd
import matplotlib.pyplot as plt
import palettable as pal
from sklearn import decomposition
from sklearn import preprocessing
from sklearn import cross_decomposition
from sklearn import linear_model
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers import BatchNormalization
from keras.optimizers import RMSprop, SGD
from keras import backend as K
from keras.regularizers import WeightRegularizer, l1, activity_l2
%matplotlib inline

In [2]:
properties = pd.DataFrame.from_csv("create-sim-data/default_parameters_sim_species_prop.csv")

In [3]:
concentration = pd.DataFrame.from_csv("create-sim-data/default_parameters_sim_gene_counts.csv")
for i in range(concentration.shape[0]):
        concentration.iloc[i,:] = concentration.iloc[i,:] / concentration.iloc[i,:].sum()
log_concentration = np.array(np.log(concentration + 1))

In [4]:
pca = decomposition.PCA(n_components=10)
pca.fit(log_concentration)
X = pca.transform(log_concentration)

In [5]:
lm = linear_model.LinearRegression()
lm.fit(X, preprocessing.scale(np.array(properties)))

/Users/davidharris/anaconda/lib/python2.7/site-packages/scipy/linalg/basic.py:884: RuntimeWarning: internal gelsd driver lwork query error, required iwork dimension not returned. This is likely the result of LAPACK bug 0038, fixed in LAPACK 3.2.2 (released July 21, 2010). Falling back to 'gelss' driver.
  warnings.warn(mesg, RuntimeWarning)


LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [6]:
print lm.score(X, preprocessing.scale(np.array(properties)))

0.355032376547


In [7]:
R2 = 1 - ((lm.predict(X) - preprocessing.scale(np.array(properties))) ** 2).sum(axis = 0) / ((preprocessing.scale(np.array(properties))) ** 2).sum(axis = 0)

In [8]:
R2

array([ 0.29943101,  0.47136872,  0.53934924,  0.26254096,  0.12175217,
        0.27835758,  0.68822528,  0.40948816,  0.27152519,  0.20828545])

In [55]:
x = preprocessing.scale(np.array(concentration))
y = np.array(concentration)

ae = Sequential()
ae.add(Dense(10, input_dim=concentration.shape[1],activity_regularizer = activity_l2(.01)))
ae.add(Activation('relu'))
ae.add(Dense(concentration.shape[1], W_regularizer=l1(0.01),activity_regularizer = activity_l2(.01)))
ae.add(Activation('softmax'))
ae.compile(optimizer=SGD(lr=.05, momentum=0.9, nesterov=True), loss='categorical_crossentropy')

In [56]:
history = ae.fit(x, y, batch_size=100, nb_epoch=500, verbose=1)

Epoch 1/500
100/100 [==============================] - 0s - loss: 11.1422
Epoch 2/500
100/100 [==============================] - 0s - loss: 11.0153
Epoch 3/500
100/100 [==============================] - 0s - loss: 10.5359
Epoch 4/500
100/100 [==============================] - 0s - loss: 10.2397
Epoch 5/500
100/100 [==============================] - 0s - loss: 9.9836
Epoch 6/500
100/100 [==============================] - 0s - loss: 9.6858
Epoch 7/500
100/100 [==============================] - 0s - loss: 9.3637
Epoch 8/500
100/100 [==============================] - 0s - loss: 9.0118
Epoch 9/500
100/100 [==============================] - 0s - loss: 8.6218
Epoch 10/500
100/100 [==============================] - 0s - loss: 8.1876
Epoch 11/500
100/100 [==============================] - 0s - loss: 7.6935
Epoch 12/500
100/100 [==============================] - 0s - loss: 7.1224
Epoch 13/500
100/100 [==============================] - 0s - loss: 6.4980
Epoch 14/500
100/100 [=====================

In [58]:
get_code = K.function([ae.layers[0].input], [ae.layers[2].output])
code = get_code([x])[0]
lm3 = linear_model.LinearRegression()
lm3.fit(code, preprocessing.scale(np.array(properties)))
print lm3.score(code, preprocessing.scale(np.array(properties)))

0.984874606073


In [60]:
pd.DataFrame(code).describe()

,0,1,2,3,4,5,6,7,8,9,...,990,991,992,993,994,995,996,997,998,999
count,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,...,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000
mean,-0.019584,-0.023591,-0.024399,-0.020890,-0.021290,-0.020523,-0.022086,-0.017778,-0.020909,-0.020664,...,-0.021996,-0.018861,-0.022553,-0.021599,-0.020386,-0.022692,-0.021075,-0.024793,-0.020288,-0.020635
std,0.002428,0.003309,0.002985,0.001333,0.002094,0.001550,0.001409,0.003050,0.001881,0.001839,...,0.002497,0.002508,0.002570,0.002998,0.002090,0.001718,0.002115,0.002617,0.000546,0.002257
min,-0.022534,-0.031070,-0.032318,-0.023364,-0.026361,-0.024123,-0.026531,-0.027276,-0.024921,-0.026234,...,-0.028916,-0.027100,-0.028304,-0.028477,-0.026433,-0.028109,-0.026678,-0.030536,-0.021555,-0.024127
25%,-0.021287,-0.026491,-0.026719,-0.021755,-0.022364,-0.020888,-0.022728,-0.019687,-0.021940,-0.021016,...,-0.024101,-0.020215,-0.024914,-0.023527,-0.022032,-0.023230,-0.022787,-0.026259,-0.020520,-0.022078
50%,-0.020820,-0.022601,-0.022766,-0.021344,-0.022207,-0.020216,-0.021489,-0.016382,-0.021722,-0.020882,...,-0.021133,-0.017580,-0.021650,-0.020183,-0.019445,-0.023104,-0.020078,-0.024263,-0.020406,-0.021749
75%,-0.018221,-0.020785,-0.022000,-0.020262,-0.019862,-0.020021,-0.021273,-0.015451,-0.019278,-0.020018,...,-0.019754,-0.017089,-0.020225,-0.019460,-0.018936,-0.022037,-0.019782,-0.023942,-0.020216,-0.019033
max,-0.012883,-0.013729,-0.021449,-0.015433,-0.016184,-0.016331,-0.018315,-0.012097,-0.016945,-0.016361,...,-0.018580,-0.012261,-0.018146,-0.015722,-0.016514,-0.017536,-0.016601,-0.016858,-0.018449,-0.013636


In [59]:
pd.DataFrame(ae.layers[2].get_weights()[0].flatten()).describe()

,0
count,10000.000000
mean,0.000777
std,0.031445
min,-0.664940
25%,-0.000455
50%,-0.000014
75%,0.000382
max,1.341285
